In [5]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nProject: Bangkit Capstone Project\nGroup: B21-CAP0194\nPart: Modelling\n'

In [6]:
vocab_size = 20000
embedding_dim = 32
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
#Import Labelled Dataset
url = 'https://drive.google.com/file/d/1nOn8OONtZA7pcX4Pvh8W_wOUg49ZWTvT/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,the ironies of the movement are too painfully...,private places
1,1,women in kuwait have launched a social media c...,public places
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places


In [ ]:
df_cut = df[:100]

In [ ]:
df_cut.head(50)

,index,text,label
0,0,the ironies of the movement are too painfully...,work places
1,1,women in kuwait have launched a social media c...,cyber
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places
5,5,this is so messed up are there any men in the...,public places
6,6,to tackle an epidemic of male violence against...,cyber
7,7,there has been a new series of harassment alle...,private places
8,8,want to see what the movement looks like its ...,public places
9,9,female students dont want to have sex w dolan ...,educational places


In [9]:
import getpass

APIKEY = getpass.getpass()

··········


In [ ]:
# running Translate API
#from googleapiclient.discovery import build
#service = build('translate', 'v2', developerKey=APIKEY)

#df_cut['indo_text'] = df_cut['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [10]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

df['indo_text'] = df['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
df['indo_text'].head(50)

In [13]:
#Save DataFrame to CSV
df_translated = pd.DataFrame()
df_translated['index'] = df['index']
df_translated['text'] = df['text']
df_translated['indo_text'] = df['indo_text']
df_translated['label'] = df['label']
df_translated.to_csv("capstonetweettranslatednew.csv", index=False)

In [14]:
#Import Translated Dataset
url = 'https://drive.google.com/file/d/1fYLD9Po3eGJacbeWnHEo_P15w9Hd8tkS/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,indo_text,label
0,0,the ironies of the movement are too painfully...,ironi gerakan ini terlalu menyakitkan jelas di...,private places
1,1,women in kuwait have launched a social media c...,perempuan di kuwait telah meluncurkan kampanye...,public places
2,2,triger warning raising awareness of sexual ass...,peringatan triger meningkatkan kesadaran akan ...,private places
3,3,charges brought against the guy that smacked m...,dakwaan yang diajukan terhadap pria yang memuk...,private places
4,4,and the fourth presenter on 26th may is hannah...,dan presenter keempat pada tanggal 26 mei adal...,educational places


In [16]:
df['text'] = df['indo_text']
df.drop(['indo_text'], axis=1)

,index,text,label
0,0,ironi gerakan ini terlalu menyakitkan jelas di...,private places
1,1,perempuan di kuwait telah meluncurkan kampanye...,public places
2,2,peringatan triger meningkatkan kesadaran akan ...,private places
3,3,dakwaan yang diajukan terhadap pria yang memuk...,private places
4,4,dan presenter keempat pada tanggal 26 mei adal...,educational places
...,...,...,...
14103,14103,Itu peta jalan yang diuji pada implementasi la...,public places
14104,14104,serbia saya pikir saya adalah satu-satunya ber...,neutral
14105,14105,jangan bergeming bersikeras untuk memperjuangk...,public places
14106,14106,pelatihan antiharassment online gratis yang di...,private places


In [17]:
labels = pd.get_dummies(df.label)

In [18]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [19]:
tweet = df_new['text'].values
label = df_new[["cyber", "educational places", "neutral", "private places", "public places", "work places"]].values

In [20]:
label

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [21]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [22]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [25]:
class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('val_accuracy')>0.90):
			print("Desired accuracy reached!")
			self.model.stop_training=True

In [26]:
import tensorflow as tf
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

'''
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])


transfer learning models for NLP: BERT, coba buka ini https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 32)            640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 256)           164864    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 64)            41216     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32)                10368     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1

In [28]:
num_epochs = 100
callbacks = myCallback()
model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), callbacks=[callbacks])

Epoch 1/100
353/353 [==============================] - 77s 102ms/step - loss: 1.3011 - accuracy: 0.4549 - val_loss: 0.9683 - val_accuracy: 0.6233
Epoch 2/100
353/353 [==============================] - 33s 95ms/step - loss: 0.8127 - accuracy: 0.7130 - val_loss: 0.8589 - val_accuracy: 0.7002
Epoch 3/100
353/353 [==============================] - 33s 94ms/step - loss: 0.6269 - accuracy: 0.7890 - val_loss: 0.8225 - val_accuracy: 0.7225
Epoch 4/100
353/353 [==============================] - 33s 94ms/step - loss: 0.4905 - accuracy: 0.8425 - val_loss: 0.8737 - val_accuracy: 0.7229
Epoch 5/100
353/353 [==============================] - 33s 94ms/step - loss: 0.3823 - accuracy: 0.8825 - val_loss: 0.8618 - val_accuracy: 0.7261
Epoch 6/100
353/353 [==============================] - 33s 94ms/step - loss: 0.2936 - accuracy: 0.9088 - val_loss: 0.9409 - val_accuracy: 0.7275
Epoch 7/100
353/353 [==============================] - 33s 94ms/step - loss: 0.2411 - accuracy: 0.9271 - val_loss: 1.0347 - val_a

In [47]:
text = ["ada orang mesum di jalan"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [48]:
prediction = model.predict(tokenized_text)

In [49]:
labels = ["cyber", "educational places", "neutral", "private places", "public places", "work places"]
predicted = prediction.argmax(axis=1)
predict_labels = [labels[predict] for predict in predicted]
print(prediction)
print(predict_labels)

[[8.1594671e-06 2.1882171e-14 4.9783659e-05 6.7176193e-04 9.9926955e-01
  6.6684640e-07]]
['public places']


In [50]:
#Exporting the Model (TFLITE)
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [51]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [52]:
import pathlib

tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

4156772

In [53]:
from google.colab import files
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
#Exporting the Tokenized Word
import json

with open( 'word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )